## Necessary Modules


<p><b>OpenCv</b></p>
<p><b>Numpy</b></p>
<p><b>Mediapipe</b></p>

<p>for opencv pip install opencv-python, this download numpy also.</p>
<p>for mediapipe pip install mediapipe.</p>


In [1]:
# importing Modules
import cv2   
import numpy as np


class Particle():
    def __init__(self):
        self.particles = []   # List of particles
        
    def emit(self):
        if self.particles:   
            self.delete_particles()
            for particle in self.particles:
                particle[0][1] += particle[2][1]   # Move particles in Y direction
                particle[0][0] += particle[2][0]   # Move paticles in X direction
                particle[1] -= 0.2                 # Decreasing radius of particle
                cv2.circle(img,tuple(particle[0]),int(particle[1]),(0,255,255))  # Draw the particle
            
    def add_particles(self,cx,cy):
        pos_x = cx                   # X position
        pos_y = cy                   # Y positon
        radius = 10                  # Radius
        direction_x = np.random.randint(-3,3)        
        direction_y = np.random.randint(-3,3)        
        particle_circle = [[pos_x,pos_y],radius,[direction_x,direction_y]]       
        self.particles.append(particle_circle)
    
    def delete_particles(self):
        particle_copy = [particle for particle in self.particles if particle[1]>0]  # Remove particles if radius is less than 0
        self.particles = particle_copy
        

In [2]:
# importing modules
import cv2
import mediapipe as mp
import numpy as np

class HandDetector():
    
    def __init__(self):
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(max_num_hands=1)
        self.mpDraw = mp.solutions.drawing_utils     # For Drawing in image
        
    def findHands(self,img,draw=True):
        imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)    # Convert BGR to RGB
        self.results = self.hands.process(imgRGB)      # Landmarks
        
        if self.results.multi_hand_landmarks:  
            for handLms in self.results.multi_hand_landmarks:   
                if draw:
                    self.mpDraw.draw_landmarks(img,handLms,self.mpHands.HAND_CONNECTIONS) # To Draw all connections
        return img
    
    def findPosition(self,img):
        lmList = []
        
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                for ids,lms in enumerate(handLms.landmark):
                    h,w,c = img.shape                   # Dimension of image
                    cx, cy = int(lms.x * w), int(lms.y * h)     # centre of each points
                    lmList.append([ids, cx, cy])     # Landmarks
        
        return lmList
         

Start by running both upper sections. As they contains classes for below code.

Below project triggers particle system on detection of tip of index finger and stops when index finger and thumb tips are pressed against each other.

Press Q to exit.

In [3]:
# importing modules
import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)   # Video feed
detector = HandDetector()   # Hand detection

particle1 = Particle()     # Particles 
while True:
    ret,frame = cap.read()
    img = detector.findHands(frame)
    lmList = detector.findPosition(img)
    
    if len(lmList)!=0:
        
        if abs(lmList[4][1]-lmList[8][1])>10:    #stop particle system if index finger and thumb are pressed
            for i in range(2):
                particle1.add_particles(lmList[8][1],lmList[8][2])  # generate particle system if index finger is detected
    particle1.emit()
    cv2.imshow('Particle System',img)
    if cv2.waitKey(2) == ord('q'):  # Press Q to exit
        break
cap.release()
cv2.destroyAllWindows()